In [24]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
#!pip install scikit-learn==0.24.0

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LogisticRegression

from sklearn.cluster import k_means
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN
!pip install hdbscan
!pip install fa_kit
!pip install kneed
from fa_kit.rotation import VarimaxRotatorPython

from sklearn import metrics
import os

from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets, linear_model

import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.spatial import distance
from scipy.cluster import hierarchy

import seaborn as sns; sns.set();
from seaborn import heatmap

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets, linear_model

import random as random
from sklearn.multiclass import OneVsRestClassifier

from xgboost import XGBClassifier

import itertools as it
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import accuracy_score

from sklearn.neighbors import LocalOutlierFactor

In [3]:
'''
def read_file(folder):
    df = pd.DataFrame()
    path = folder+'/'
    df_new = pd.read_csv(path+'benign_traffic.csv')
    df_new['label'] = 'benign_traffic'
    df = pd.concat([df,df_new], ignore_index=True)
    
    directory = os.fsencode(path+'gafgyt_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'gafgyt_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'gafgyt_'+filename
            df = pd.concat([df,df_new], ignore_index=True)
    directory = os.fsencode(path+'mirai_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'mirai_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'mirai_'+filename
            df = pd.concat([df,df_new], ignore_index=True)
    return df'''

In [21]:
def read_file(folder, samp = False, lof=False):
    df = pd.DataFrame()
    path = folder+'/'
    df_new = pd.read_csv(path+'benign_traffic.csv')
    df_new['label'] = 'benign_traffic'
    if samp==True:
        df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
    if lof==True:
        #df_new = pd.DataFrame(scaler.fit_transform(df_new))
        df_new = filter_lof(df_new)
        #df_new = scaler.inverse_transform(df_new)
    df = pd.concat([df,df_new], ignore_index=True)
    
    directory = os.fsencode(path+'gafgyt_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'gafgyt_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'gafgyt_'+filename
            if samp==True:
                df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
            if lof==True:
                #df_new = pd.DataFrame(scaler.fit_transform(df_new))
                df_new = filter_lof(df_new)
                #df_new = scaler.inverse_transform(df_new)
            df = pd.concat([df,df_new], ignore_index=True)
    directory = os.fsencode(path+'mirai_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'mirai_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'mirai_'+filename
            if samp==True:
                df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
            if lof==True:
                #df_new = pd.DataFrame(scaler.fit_transform(df_new))
                df_new = filter_lof(df_new)
                #df_new = scaler.inverse_transform(df_new)
            df = pd.concat([df,df_new], ignore_index=True)
    return df

In [4]:
def pca_func(rate, data):    
    df_wo_label = data.drop(columns = ['label'])
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(df_wo_label)
    pca = PCA(n_components=rate, random_state=0)
    pca_comp = pca.fit(data_scaled).components_
    return pca_comp, data_scaled

In [5]:
# LEts transform our original dataset to lower dimensional
def decrease_dim(components, data_scaled):
    varimax = VarimaxRotatorPython()
    rotated_weights = varimax.rotate(components.T)
    df_lowdim = pd.DataFrame(np.dot(data_scaled, rotated_weights))
    #df_lowdim["label"] = df["label"]
    rotated_data = np.dot(data_scaled, rotated_weights)
    return df_lowdim, rotated_data

In [6]:
import scipy.special

def adj_rand(data):
    tab = pd.crosstab(data['label'], data['cluster_label'])
    df_tab = pd.DataFrame(tab)
    a = list(df_tab.sum(axis=1))
    b = list(df_tab.sum(axis=0))
    n = df_tab.values.tolist()
    sum_n = 0
    for i in range(len(n)):
        for j in range(len(n[0])):
            sum_n += scipy.special.comb(n[i][j],2)
    sum_a = 0
    for k in range(len(a)):
        sum_a += scipy.special.comb(a[k],2)
    sum_b = 0
    for l in range(len(b)):
        sum_b += scipy.special.comb(b[l],2)
    
    ari = (sum_n - sum_a*sum_b/scipy.special.comb(sum(a),2))/(0.5*(sum_a+sum_b)-sum_a*sum_b/scipy.special.comb(sum(a),2))
    return(ari)

In [7]:
def evall(df_cl):
    sc = []
    sc.append(metrics.rand_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(adj_rand(df_cl))
    sc.append(metrics.mutual_info_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(metrics.adjusted_mutual_info_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(metrics.homogeneity_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(metrics.completeness_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(metrics.v_measure_score(df_cl['label'], df_cl['cluster_label']))
    sc.append(metrics.fowlkes_mallows_score(df_cl['label'], df_cl['cluster_label']))
    return sc

In [891]:
folder = '/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/Philips_B120N10_Baby_Monitor/alex2/baby_k10_e1_lof'
#folder = '/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/SimpleHome_XCS7_1003_WHT_Security_Camera/attila1/c3_e10'
df = read_file(folder)
df = df_original.sample(n=100000, replace=False, random_state=0)#.reset_index(drop=True)
#df = df.dropna().reset_index(drop=True)

## LOF

In [22]:
def filter_lof(df, k=20) :
    lof = LocalOutlierFactor(n_neighbors=k)
    scaler = StandardScaler()
    df2 = pd.DataFrame.copy(df)
    df2 = df2.drop(columns = ['label'])
    df2 = pd.DataFrame(scaler.fit_transform(df2))
    df2["_lof"] = lof.fit_predict(df2)
    df2 = df2[df2["_lof"]>0].drop(columns="_lof")#.reset_index(drop=True)
    df2 = pd.DataFrame(scaler.inverse_transform(df2))
    df2['label'] = df['label']
    return df2.reset_index(drop=True)

# ALGORITHMS

In [8]:
def kmeans(low_data, num_cl):
    low_data_2 = low_data.copy()
    kmeans = KMeans(init="random",n_clusters=num_cl,n_init=10,max_iter=200,random_state=43)
    kmeans.fit(low_data_2)
    low_data_2["cluster_label"] = kmeans.labels_
    low_data_2["label"] = df["label"]
    return low_data_2

In [9]:
def minibatch(low_data, num_cl):
    low_data_2 = low_data.copy()
    mini_kmeans = MiniBatchKMeans(init="random",n_clusters=num_cl,n_init=10,max_iter=200,random_state=43)
    mini_kmeans.fit(low_data_2)
    low_data_2["cluster_label"] = mini_kmeans.labels_
    low_data_2["label"] = df["label"]
    return low_data_2

In [10]:
def mean_shift(low_data):
    low_data_2 = low_data.copy()
    mean_sh = MeanShift().fit(low_data_2)
    low_data_2["cluster_label"] = mean_sh.labels_
    low_data_2["label"] = reduced_label
    return low_data_2

In [11]:
def optics(low_data):
    low_data_2 = low_data.copy()
    opti = OPTICS().fit(low_data_2)
    low_data_2["cluster_label"] = opti.labels_
    low_data_2["label"] = reduced_label
    return low_data_2

# DBSCAN

In [12]:
def dbscan(low_data, ep, min_samp):
    low_data_2 = low_data.copy()
    dbsc = DBSCAN(eps=ep, min_samples=min_samp).fit(low_data_2)
    low_data_2["cluster_label"] = dbsc.labels_
    low_data_2["label"] = reduced_label
    return low_data_2

In [834]:
df_reduced = df.sample(n=100000)
reduced_label = df_reduced['label'].reset_index(drop=True)
comp, df_scaled = pca_func(0.7, df_reduced)
df_low, np_low = decrease_dim(comp, df_scaled)

In [835]:
eval_dbscan = []
for j in [0.18,0.2,0.22,0.25,0.3,0.35,0.4,0.5]:
    df_dbscan = dbscan(df_low, j, 5)
    metrics_dbscan = evall(df_dbscan)
    eval_dbscan.append(metrics_dbscan)
eval_dbscan

[[0.9081935389353893,
  0.5725643034768708,
  1.7076800468607916,
  0.7596554061716501,
  0.7584602111532377,
  0.7616596325818332,
  0.7600565549298826,
  0.6252566422872781],
 [0.9090739311393113,
  0.5775966605445773,
  1.717163139704883,
  0.7645584847554555,
  0.7626720941778928,
  0.7671809190968817,
  0.7649198623709206,
  0.6298605609915215],
 [0.9098312971129712,
  0.5822506636869994,
  1.7238507826399327,
  0.7693890136370647,
  0.7656423877536477,
  0.7737515020454022,
  0.7696755865795288,
  0.6341765113233925],
 [0.8968094640946409,
  0.5449844603476173,
  1.595296795586191,
  0.7358106787276452,
  0.708545577174517,
  0.7658832944021328,
  0.7360995587399127,
  0.6065291289845909],
 [0.8947756793567936,
  0.5458167235394826,
  1.5504960882356278,
  0.7308593582362228,
  0.6886474973091542,
  0.7791067714589555,
  0.7310895832067802,
  0.6098601071385136],
 [0.8952224556245563,
  0.5487974247746159,
  1.5533347459592046,
  0.7333073256065497,
  0.6899082773600623,
  0.7830

In [836]:
eval_dbscan = []
#for i in [3,4,5,6,9,10, 11, 12]:
for i in [3,5,8,10,15,20]:
    df_dbscan = dbscan(df_low, 0.22, i)
    metrics_dbscan = evall(df_dbscan)
    eval_dbscan.append(metrics_dbscan)
eval_dbscan

[[0.9101824386243862,
  0.5840075335335431,
  1.731209411718691,
  0.7692871761350932,
  0.7689106975140932,
  0.7708568811384351,
  0.7698825593904114,
  0.6357457505888556],
 [0.9098312971129712,
  0.5822506636869994,
  1.7238507826399327,
  0.7693890136370647,
  0.7656423877536477,
  0.7737515020454022,
  0.7696755865795288,
  0.6341765113233925],
 [0.9091912619126191,
  0.5785958353478443,
  1.7137446525617779,
  0.7663523091637212,
  0.7611537848873731,
  0.7719856892713004,
  0.7665314724091745,
  0.6308284591857392],
 [0.9087333721337213,
  0.5759008865486979,
  1.7087180851850665,
  0.7636931507462328,
  0.7589212522997113,
  0.7688878935446882,
  0.7638720643009572,
  0.628348686728752],
 [0.9141379257792578,
  0.5911574344616732,
  1.7702650376686504,
  0.7796420091525377,
  0.7862571192628173,
  0.7733671680587154,
  0.7797588773570178,
  0.6399522969529821],
 [0.9133131785317853,
  0.5869293256513347,
  1.7599976633475465,
  0.7757371067299358,
  0.7816969003214007,
  0.770

## Building tree

In [17]:
import scipy.special

def adj_rand_2(label_pred, label_true):
    tab = pd.crosstab(label_true, label_pred)
    df_tab = pd.DataFrame(tab)
    a = list(df_tab.sum(axis=1))
    b = list(df_tab.sum(axis=0))
    n = df_tab.values.tolist()
    sum_n = 0
    for i in range(len(n)):
        for j in range(len(n[0])):
            sum_n += scipy.special.comb(n[i][j],2)
    sum_a = 0
    for k in range(len(a)):
        sum_a += scipy.special.comb(a[k],2)
    sum_b = 0
    for l in range(len(b)):
        sum_b += scipy.special.comb(b[l],2)
    
    ari = (sum_n - sum_a*sum_b/scipy.special.comb(sum(a),2))/(0.5*(sum_a+sum_b)-sum_a*sum_b/scipy.special.comb(sum(a),2))
    return(ari)

In [18]:
def evall_2(label_pred, label_true):
    sc = []
    sc.append(metrics.rand_score(label_true, label_pred))
    sc.append(adj_rand_2(label_true, label_pred))
    sc.append(metrics.mutual_info_score(label_true, label_pred))
    sc.append(metrics.adjusted_mutual_info_score(label_true, label_pred))
    sc.append(metrics.homogeneity_score(label_true, label_pred))
    sc.append(metrics.completeness_score(label_true, label_pred))
    sc.append(metrics.v_measure_score(label_true, label_pred))
    sc.append(metrics.fowlkes_mallows_score(label_true, label_pred))
    return sc

In [19]:
#read in original data (wo anonymization)
def read_file_o(dfs, device):
    df = pd.DataFrame()
    for i in dfs:
        #df_new = pd.read_csv(i+'.csv')
        #df_new = pd.read_csv('./Simple_home_1003_sec_cam/'+i+'.csv')
        df_new = pd.read_csv('./'+device+'/'+i+'.csv')
        df_new['label'] = i
        df = pd.concat([df,df_new], ignore_index=True)
    return df

In [866]:
files = ['benign_traffic', 'bashlite_combo', 'bashlite_junk', 'bashlite_scan', 'bashlite_tcp', 'bashlite_udp',
       'mirai_ack', 'mirai_scan', 'mirai_syn', 'mirai_udp', 'mirai_udpplain']
df_original = read_file_o(files)
df_original = df.dropna().reset_index(drop=True)

In [ ]:
df_eval = pd.DataFrame(columns = ['rand_score', 'adjusted_rand_score', 'mutual_info_score', 'adjusted_mutual_info_score', 
                                  'homogeneity_score', 'completeness_score', 'v_measure_score', 'fowlkes_mallows_score'])
#for device in ['SimpleHome_XCS7_1003_WHT_Security_Camera', 'Philips_B120N10_Baby_Monitor']:
for device in ['SimpleHome_XCS7_1003_WHT_Security_Camera']:
    print(device)
    #original file beolvasas
    files = ['benign_traffic', 'bashlite_combo', 'bashlite_junk', 'bashlite_scan', 'bashlite_tcp', 'bashlite_udp', 'mirai_ack', 'mirai_scan', 'mirai_syn', 'mirai_udp', 'mirai_udpplain']
    #scaling
    #df_o_wo_label = df_original.drop(columns = ['label'])
    #scaler = StandardScaler()
    #data_o_scaled = scaler.fit_transform(df_o_wo_label)
    #for algo_anon in ['alex2', 'attila1', 'attila2']:
    for algo_anon in ['alex1b']:
        print(algo_anon)
        if algo_anon=='alex1b' or 'alex2':
            df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device, samp=True, lof=True)
        if algo_anon=='attila1' or 'attila2':
            df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device, samp=True)
        #scaling
        df_o_wo_label = df_original.drop(columns = ['label'])
        scaler = StandardScaler()
        data_o_scaled = scaler.fit_transform(df_o_wo_label)
        for file in os.listdir('/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo_anon):
            if file.startswith('c') or file.startswith('k'):
                print(file)
                #file beolvasas
                folder = '/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo_anon+'/'+file
                df = read_file(folder)
                df = df.dropna().reset_index(drop=True)
                for seed in [0,1,2,3]:
                    print(seed)
                    df_reduced = df.sample(n=100000,replace=False, random_state=seed)
                    df_reduced2 = df_reduced.copy()
                    reduced_label = df_reduced['label'].reset_index(drop=True)
                    comp, df_scaled = pca_func(0.7, df_reduced)
                    df_low, np_low = decrease_dim(comp, df_scaled)
                    df_reduced2 = df_reduced.reset_index(drop=True) 
                    ultim_best_scores = [0,0,0,0,0,0,0,0]
                    df_ultim_best = pd.DataFrame()
                    for algo in ['optics', 'kmeans', 'minibatch', 'dbscan']:
                        print(algo)
                        if algo is 'meanshift':
                            df_meanshift = mean_shift(df_low)
                            metrics_meanshift = evall(df_meanshift)
                            if metrics_meanshift[6]>ultim_best_scores[6]:
                                ultim_best_scores = metrics_meanshift
                                df_ultim_best = df_meanshift
                        if algo is 'optics':
                            df_optics = optics(df_low)
                            metrics_optics = evall(df_optics)
                            if metrics_optics[6]>ultim_best_scores[6]:
                                ultim_best_scores = metrics_optics
                                df_ultim_best = df_optics
                        if algo is 'kmeans':
                            eval_kmeans = [0,0,0,0,0,0,0,0]
                            df_best = pd.DataFrame()
                            for k in range(7,15):
                                df_kmeans = kmeans(df_low, k)
                                metrics_kmeans = evall(df_kmeans)
                                if metrics_kmeans[6]>eval_kmeans[6]:
                                    eval_kmeans=metrics_kmeans
                                    df_best_kmeans = df_kmeans
                            if eval_kmeans[6]>ultim_best_scores[6]:
                                ultim_best_scores = eval_kmeans
                                df_ultim_best = df_best_kmeans
                        if algo is 'minibatch':
                            eval_minibatch = [0,0,0,0,0,0,0,0]
                            df_best = pd.DataFrame()
                            for k in range(7,15):
                                df_minibatch = minibatch(df_low, k)
                                metrics_minibatch = evall(df_minibatch)
                                if metrics_minibatch[6]>eval_minibatch[6]:
                                    eval_minibatch=metrics_minibatch
                                    df_best_minibatch = df_minibatch
                            if eval_minibatch[6]>ultim_best_scores[6]:
                                ultim_best_scores = eval_minibatch
                                df_ultim_best = df_best_minibatch
                        if algo is 'dbscan':
                            eval_dbscan = [0,0,0,0,0,0,0,0]
                            df_best = pd.DataFrame()
                            for ep in [0.18,0.2,0.22,0.25,0.3,0.4,0.5]:
                                for min_sam in [3,5,8,10,12]:
                                    df_dbscan = dbscan(df_low, ep, min_sam)
                                    metrics_dbscan = evall(df_dbscan)
                                    if metrics_dbscan[6]>eval_dbscan[6]:
                                        eval_dbscan=metrics_dbscan
                                        df_best_dbscan = df_dbscan
                            if eval_dbscan[6]>ultim_best_scores[6]:
                                ultim_best_scores = eval_dbscan
                                df_ultim_best = df_best_dbscan
                    df_reduced2["cluster_label"] = df_ultim_best["cluster_label"]
                    ###
                    df_eval.loc[device+'_'+algo_anon+'_'+file+'_'+str(seed)] = evall(df_ultim_best)  
                    y = df_reduced2['cluster_label'][:190000]
                    X = df_low.iloc[:190000,:]
                    dt = OneVsRestClassifier(XGBClassifier(random_state=4, max_depth=8), n_jobs=7)
                    dt.fit(X, y)  
                    test_label = dt.predict(df_low.iloc[90000:,:])
                    true_label = df_reduced2['cluster_label'][90000:]    
                    #apply same tree to original
                    #decrease dim
                    varimax = VarimaxRotatorPython()
                    rotated_weights = varimax.rotate(comp.T)
                    df_o_lowdim = pd.DataFrame(np.dot(data_o_scaled, rotated_weights))
                    #test original
                    test_label_o = dt.predict(df_o_lowdim)
                    true_label_o = df_original['label']
                    ###
                    df_eval.loc[device+'_'+algo_anon+'_'+file+'_'+str(seed)+'_original'] = evall_2(test_label_o, true_label_o)  
                    df_eval.to_csv('eval_total_'+device+'.csv')
                            

SimpleHome_XCS7_1003_WHT_Security_Camera
alex1b
k10_e10_lof
0
optics


/opt/conda/lib/python3.7/site-packages/sklearn/cluster/_optics.py:803: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


kmeans
minibatch
dbscan
1
optics


/opt/conda/lib/python3.7/site-packages/sklearn/cluster/_optics.py:803: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


kmeans
minibatch
dbscan
2
optics


...

kmeans
minibatch
dbscan
k20_e10_lof
0
optics


/opt/conda/lib/python3.7/site-packages/sklearn/cluster/_optics.py:803: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


kmeans
minibatch
dbscan
1
optics


/opt/conda/lib/python3.7/site-packages/sklearn/cluster/_optics.py:803: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


kmeans
minibatch
dbscan


In [907]:
folder = '/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+'SimpleHome_XCS7_1003_WHT_Security_Camera'+'/'+'alex2'+'/'+'k10_e10_lof'
df = read_file(folder)
df = df.dropna().reset_index(drop=True)

comp, df_scaled = pca_func(0.7, df)
df_low, np_low = decrease_dim(comp, df_scaled)

df_kmeans = kmeans(df_low, 8)
print(df_kmeans)
print(evall(df_kmeans))

               0         1         2         3         4  cluster_label  \
0       2.753317 -0.648823 -0.251951 -0.005992 -5.147781              6   
1      -2.246141 -3.187886 -2.022540  0.859386 -5.003501              6   
2       2.757108 -0.626852 -0.252058 -0.005805 -5.143632              6   
3       2.751079 -0.639435 -0.251551 -0.005815 -5.129953              6   
4       2.757136 -0.626673 -0.252192 -0.005809 -5.143990              6   
...          ...       ...       ...       ...       ...            ...   
396037 -4.715749  6.540347 -0.266300  0.015813  1.186763              4   
396038 -4.642063  6.098272 -0.262651  0.011855  0.343731              4   
396039 -4.281696  5.677139 -0.299778  0.015696  0.333500              4   
396040  6.934503  3.852830  0.300300 -0.056403  0.548733              7   
396041 -4.387097  5.896750 -0.276218  0.011317 -0.622355              4   

                 label  
0       benign_traffic  
1       benign_traffic  
2       benign_traffic  

In [827]:
df_reduced = df.sample(n=100000)
df_reduced2 = df_reduced.copy()
reduced_label = df_reduced['label'].reset_index(drop=True)
comp, df_scaled = pca_func(0.7, df_reduced)
df_low, np_low = decrease_dim(comp, df_scaled)

In [828]:
df_reduced2 = df_reduced.reset_index(drop=True)
df_dbscan = dbscan(df_low, 0.20, 6)
df_reduced2["cluster_label"] = df_dbscan["cluster_label"]
evall(df_dbscan)

[0.827229472294723,
 0.3943155857846049,
 1.2840817877990083,
 0.6241371955234212,
 0.5747797770717933,
 0.6890014119045573,
 0.6267288695085382,
 0.5102440897307984]

In [829]:
y = df_reduced2['cluster_label'][:190000]
X = df_low.iloc[:190000,:]
dt = OneVsRestClassifier(XGBClassifier(random_state=4, max_depth=8), n_jobs=7)
dt.fit(X, y)

test_label = dt.predict(df_low.iloc[90000:,:])
true_label = df_reduced2['cluster_label'][90000:]

print(metrics.accuracy_score(test_label, true_label))

0.9992
